In [16]:
import torch

x = torch.randn(1,10)
print(x)
print(x.shape)

tensor([[-1.9310,  1.9308, -0.8158,  0.1399, -1.9489,  0.9859, -1.4549,  0.3011,
          0.2422, -1.1729]])
torch.Size([1, 10])


In [17]:
import torch.nn as nn

input_size = 10
hidden_size =100

x = nn.LSTM(input_size=input_size,
            hidden_size=100)
print(x)

LSTM(10, 100)


In [18]:
embed = nn.Embedding(num_embeddings=10,
                     embedding_dim=100)
print(embed)

Embedding(10, 100)


In [19]:
random_data = torch.randint(low=1,
                            high=10,
                            size=(1,10))
print(random_data)
print(random_data.shape)

tensor([[9, 1, 6, 8, 6, 2, 7, 2, 5, 7]])
torch.Size([1, 10])


In [20]:
embed_out = embed(random_data)

In [21]:
print(embed_out.shape)

torch.Size([1, 10, 100])


In [22]:
# if input size not 1,10
random_data2 = torch.randint(1,10,size=(4,5))
embed_out2 = embed(random_data2)
print(embed_out2.shape)

torch.Size([4, 5, 100])


In [23]:
import torch
import torch.nn as nn
# Matrix multiplication
mat1 = torch.rand(3,3)
mat2 = torch.rand(3,3)

res1 = torch.matmul(mat1,mat2) 
print(res1)

linear = nn.Linear(3,1)
res2 = linear(mat1)
print(res2)

tensor([[0.3339, 0.4873, 0.2267],
        [0.9517, 1.1873, 0.4183],
        [0.1010, 0.2022, 0.1148]])
tensor([[0.3649],
        [0.8732],
        [0.1586]], grad_fn=<AddmmBackward0>)


In [24]:
# Attention implementation

import torch
import torch.nn as nn

encoder_output = torch.rand(1,10)
decoder_output = torch.rand(1,10)

# dim0: first dim to be transposed
# dim1: second dim to be transposed
decoder_transpose = torch.transpose(decoder_output,0,1)

dot_product = torch.matmul(encoder_output, decoder_output)


attention_weights = nn.Softmax(dot_product)
print('Encoder output:',encoder_output)
print('Decoder output',decoder_output)
print('Dot product:', dot_product)
print('Attention weights:',attention_weights)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x10 and 1x10)

In [25]:
import torch
import torch.nn as nn


class Attention(nn.Module):
    def __init__(self, hidden_size):
        super().__init__()
        self.linear = nn.Linear(hidden_size, hidden_size)
        self.softmax = nn.Softmax(dim=0)
        
    def forward(self,encoder_output, decoder_output):
        
        encoder_transform = self.linear(encoder_output)
        print("Encoder transform shape",encoder_transform.shape)
        
        dot_product = torch.matmul(encoder_transform , decoder_output.T)
        print("Dot product shape:", dot_product.shape)
        
        # dim=0 for tensor, else->softmax type
        attention_weights = self.softmax(dot_product)
        print('Attention weights shape:', attention_weights.shape)
        
        # attention_weights = torch.tensor(attention_weights)
        
        # weighted sum of encoder output
        attended_encoder_output = torch.matmul(attention_weights, encoder_transform)
        # attention_weights * encoder_transform
        
        print('Attended encoder output:', attended_encoder_output.shape)
        
        return attention_weights, attended_encoder_output
    
hidden_size = 10

attention_model = Attention(hidden_size)

attention_weights, attended_encoder_output = attention_model(encoder_output, decoder_output)

# print("Encoder output shape:", encoder_output)
# print("Decoder output shape:", decoder_output)
# print("Attention weights:", attention_weights)
print("Attended encoder output:", attended_encoder_output)

Encoder transform shape torch.Size([1, 10])
Dot product shape: torch.Size([1, 1])
Attention weights shape: torch.Size([1, 1])
Attended encoder output: torch.Size([1, 10])
Attended encoder output: tensor([[ 0.1028,  0.2465, -0.1070, -0.1965, -0.2749,  0.4547,  0.3668,  0.0807,
          0.2115,  0.2449]], grad_fn=<MmBackward0>)


In [26]:
from numpy import array
from numpy import random
from numpy import dot
from scipy.special import softmax

word_1 = array([1, 0, 0])
word_2 = array([0, 1, 0])
word_3 = array([1, 1, 0])
word_4 = array([0, 0, 1])

words = array([word_1, word_2, word_3, word_4])
print("Word embded size:", words.shape)

random.seed(42)
W_Q = random.randint(3, size=(3, 3))
W_K = random.randint(3, size=(3, 3))
W_V = random.randint(3, size=(3, 3))

Q = words @ W_Q
K = words @ W_K
V = words @ W_V
print('k size:{}, q size:{}, v size{}'.format(K.shape,Q.shape,V.shape))

scores = Q @ K.transpose()
print('Scores shape:', scores.shape)

weights = softmax(scores / K.shape[1] ** 0.5, axis=1)
attention = weights @ V
print(attention)
print('Attention shape:', attention.shape)

Word embded size: (4, 3)
k size:(4, 3), q size:(4, 3), v size(4, 3)
Scores shape: (4, 4)
[[0.98522025 1.74174051 0.75652026]
 [0.90965265 1.40965265 0.5       ]
 [0.99851226 1.75849334 0.75998108]
 [0.99560386 1.90407309 0.90846923]]
Attention shape: (4, 3)


In [27]:
import torch
import torch.nn as nn
from numpy import array


word_1 = array([1, 0, 0])
word_2 = array([0, 1, 0])
word_3 = array([1, 1, 0])
word_4 = array([0, 0, 1])

words = array([word_1, word_2, word_3, word_4])
words = torch.tensor(words)
print("Words shape:",words.shape)

# initialize a single layer LSTM with 35 hidden layers
# 4 words in vocab
# 3 dimensional embedding

embed = nn.Embedding(num_embeddings=4,
                     embedding_dim=3)
print(embed)
lstm = nn.LSTM(input_size=3,hidden_size=35)
print(lstm)
soft = nn.Linear(in_features=35,out_features=3)
print(soft)
# initialize hidden and cell state of lstm

# nature of h_0 and c_0:
#     -> shape: ( D*num_layers, Hout)
#         D = 1 (2 for bidirectional)
#         num_layers = 1 for single layer
#         Hout = hidden_size

h_0 = torch.zeros(1,3,35)
c_0 = torch.zeros(1,3,35)
input_data = embed(words)
print(input_data.shape)
# print(input_data)

Words shape: torch.Size([4, 3])
Embedding(4, 3)
LSTM(3, 35)
Linear(in_features=35, out_features=3, bias=True)
torch.Size([4, 3, 3])


In [28]:
h_1, c_1 = lstm(input_data,(h_0, c_0))
print("h1 shape",h_1.shape)
print(h_1[-1].shape)
# tuple containing ltm
# print(c_1)
print('c1 shape',len(c_1))

h1 shape torch.Size([4, 3, 35])
torch.Size([3, 35])
c1 shape 2


In [29]:
# attention mechanism
# query -->> is the output of previous lstm state

# output of previous LSTM , since first Layer, init to zero
prev_h = torch.zeros(h_1.shape)

w_query = nn.Linear(35,35)

query = w_query(prev_h)

In [30]:
# size of query should be 4x3x35 
print(query.shape)


torch.Size([4, 3, 35])


In [31]:
lstm

LSTM(3, 35)

In [32]:
prev_h = torch.zeros(1,3,35)
prev_c = torch.zeros(1,3,35)


# attention mechanism
attention_weights1 = nn.Linear(35,35)

all_h = []
all_weighted_h = []
for i in range(5):
    next_h, next_c = lstm(input_data,(prev_h,prev_c))
    # saving hidden states
    all_h.append(next_h[-1])
    # compute weighted version of hidden states
    all_weighted_h.append(torch.matmul(next_h[-1],attention_weights1.weight))

In [33]:
import torch

sequence = torch.randint(1,10,size=(1,6))
print(sequence)

tensor([[9, 6, 8, 2, 3, 3]])


In [34]:
embed = torch.nn.Embedding(6,16)
embedded_sequence = embed(sequence)

# seperates tensor from computational graph 
# i.e does not require gradient
embed_detach = embedded_sequence.detach()
print(embedded_sequence.shape)
print(embed_detach.shape)

IndexError: index out of range in self

In [ ]:
# Wq = Wk = dk x d or dq x d
# since we calculate dot product between the query and key vector
# 
# Wv = dv x d
# where d = size of each word vector
#         = 6 in above sequence
# dq, dk, dv can be arbitary
# d = embed_sequence[1]

w_query = torch.nn.Linear(in_features=24,out_features=16)
w_key = torch.nn.Linear(in_features=24,out_features=16)
w_value = torch.nn.Linear(in_features=24,out_features=16)

# computation of unnormalized attention weights

# for a single input sentence
query2 = torch.matmul(embed_detach[-1],w_query.weight)
print(query2.shape)

In [ ]:
w_query2 = torch.nn.Parameter(torch.rand(24,6))
print(w_query2.shape)

# pytorch's linear layer stores weights in the format (out_feature, in_feature) so, need to Transpose during matrix multiplication
print(w_query.weight.shape)

In [ ]:
keys = torch.matmul(embed_detach[-1],w_query.weight)
values = torch.matmul(embed_detach[-1], w_value.weight)
print(keys.shape)
print(values.shape)

In [ ]:
# compute matmul between queries(output of last hidden lstm, RNN, encoder)

omega = torch.matmul(query2, keys.T)
print(omega.shape)

In [ ]:
# Computation of attention scores

import torch.nn.functional as F

attention_weights_all = F.softmax(omega / 24**0.5, dim=0)
print(attention_weights_all)
print(attention_weights_all.shape)

In [ ]:

# context vector is the attention weighted version of the original query input 

context_vector = torch.matmul(attention_weights_all,values)
print(context_vector)
print(context_vector.shape)

In [1]:
import torch

lstm = torch.nn.LSTM(input_size=10,hidden_size=100)
embed = torch.nn.Embedding(6,10)

inputs = embed.weight

prev_c = torch.zeros(1,100)
prev_h = torch.zeros(1,100)

next_h, next_c = lstm(inputs, (prev_h,prev_c))
print(next_h.size())
print(next_c)

torch.Size([6, 100])
(tensor([[-0.0391,  0.0237, -0.0291, -0.2388,  0.0705,  0.1018,  0.0432,  0.1690,
         -0.0967,  0.1817,  0.0355, -0.1110,  0.2740, -0.0509, -0.0468,  0.1163,
          0.0146, -0.0719,  0.0260, -0.0145,  0.0454,  0.0019,  0.1160, -0.1382,
         -0.0287, -0.0772, -0.1221,  0.1555,  0.0802, -0.1099,  0.1374,  0.1561,
         -0.0966,  0.0096,  0.0791,  0.0589, -0.0927,  0.0040, -0.0584, -0.0419,
         -0.0044,  0.0862, -0.0450,  0.0529, -0.0382, -0.0347,  0.0328, -0.0913,
         -0.1001, -0.1548, -0.0102,  0.0643, -0.0453, -0.0988,  0.2477,  0.1163,
         -0.0070,  0.0392,  0.0223, -0.0076, -0.1271,  0.1051, -0.1325,  0.0017,
          0.0294,  0.1162,  0.1340,  0.0202, -0.0006,  0.0756,  0.0003,  0.1321,
          0.1171, -0.0178, -0.0402, -0.0771, -0.0311, -0.1046,  0.0575,  0.0502,
          0.0278,  0.0284, -0.0339, -0.0546, -0.0714, -0.0794,  0.1206,  0.0172,
          0.0972,  0.0396, -0.1417,  0.1030,  0.0654, -0.0376, -0.0060,  0.1955,
      

In [2]:
w_attn1 = torch.nn.Linear(in_features=100,out_features=100)
w_attn2 = torch.nn.Linear(in_features=100,out_features=100)
w_v = torch.nn.Linear(in_features=100,out_features=100)

all_h, all_h_w = [],[]

for i in range(6):
    next_h, next_c = lstm(inputs, (prev_h,prev_c)) 
    prev_h, prev_c = next_h, next_c
    
    all_h.append(next_h[-1])
    all_h_w.append(w_attn1(next_h[-1]))

AttributeError: 'tuple' object has no attribute 'dim'